In [1]:
# Import the two main libraries
from bs4 import BeautifulSoup       # to process html
import requests
import urllib.request,sys,time
import pandas as pd
from tqdm import tqdm
import pickle

In [2]:
import warnings
warnings.filterwarnings("ignore")

Here we scrape all the links of the sun about the war, in this case, we download the page with requests and then we look for all the anchor tags that have a class teaser-anchor teaser-anchor--search and then we save the hypertext reference.

In [3]:
#get all the links of the articles

sun_links = []
for page in tqdm(range(1,32)):

    url = 'https://www.the-sun.com/page/'+str(page)+'/?s=ukraine+war'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    links = soup.find_all('a',{'teaser-anchor teaser-anchor--search'})
    
    for link in links:
        sun_links.append(link['href'].strip())

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:22<00:00,  1.36it/s]


Since sometimes the reply by the server is an HTML page because there is an issue loading the JSON file we need to try to parse the JSON in order to see if it's a JSON or not and then remove the ones that are HTML.

In [4]:

file_name = "sun_links.pkl"

open_file = open(file_name, "wb")
pickle.dump(sun_links, open_file)
open_file.close()

In [5]:
# to load
open_file = open("sun_links.pkl", "rb")
sun_links = pickle.load(open_file)
open_file.close()

In [6]:
#creating the dataframe

SunArticles = pd.DataFrame({'title': pd.Series([], dtype='string'),
                             'author': pd.Series([], dtype='string'),
                             'date': pd.Series([], dtype='float'),
                             'text': pd.Series([], dtype='string'),
                             })


Here we load the links that we found before and we store the data scraped on the dataset.

In [154]:
for i,link in tqdm(enumerate(sun_links)):
    
    page = requests.get(link)
    soup = BeautifulSoup(page.text, "html.parser")
    
    #get title
    title = soup.find_all("h1", {"class": "article__headline"})[0].get_text()
    
    #get author
    authors = soup.find_all('a', {'class': 'author url fn article__author-link t-p-color'}) 
    if len(authors) > 0:
        author= authors[0].get_text()
    elif len(soup.find_all('span', {'class': 'article__author-name t-p-color'})) > 0:
        author = soup.find_all('span', {'class': 'article__author-name t-p-color'})[0].get_text()
    else:
        author = ''
    #get date
    date = soup.find_all('span', {'class': 'article__timestamp'})[0].get_text()
    date = pd.to_datetime(date).strftime('%d/%m/%y')
    
    #get text
    text = ''
    for p in soup.find_all('p'):
        text += p.text
    
    SunArticles = SunArticles.append({"title": title, "author": author, "date": date, "text": text}, ignore_index=True)

304it [03:18,  1.53it/s]


In [155]:
SunArticles

,title,author,date,text
0,Ukraine news latest — Kyiv issues fierce messa...,Louis Allwood,29/04/23,UKRAINE sent a defiant message to Russia's Vla...
1,"Pentagon leak suspect Jack Teixeira, 21, ‘used...",Israel Salas-Rodriguez,27/04/23,PENTAGON leak suspect Jack Teixeria had used h...
2,China’s Xi Jinping makes pledge to Zelensky in...,Nick Parker,26/04/23,CHINA’s leader Xi Jinping pledged not to add “...
3,Schoolkids chanting ‘he’s OUR King’ drown out ...,Matt Wilkinson,26/04/23,FLAG-WAVING school children drowned out a nois...
4,Russian fighter jet bursts into flames before ...,Henry Holloway,26/04/23,A RUSSIAN fighter jet burst into flames and cr...
...,...,...,...,...
299,Putin ‘may exploit holidays & take military ac...,Chris Bradford,22/12/21,VLADIMIR Putin may exploit Christmas and take ...
300,"US detects Russia massing 175,000 troops on bo...",H. J. Hayes,03/12/21,US intelligence has detected Russia massing 17...
301,Dramatic moment US fighter jets intercept Russ...,Will Stewart,28/04/21,DRAMATIC video catches the cat-and-mouse momen...
302,Biden warns Putin the US will act firmly in de...,Mollie Mansfield,13/04/21,JOE Biden warned Vladimir Putin that the Unite...


In [156]:
SunArticles.to_csv("SunArticles.csv")